# preprocessing

In [1]:
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt

In [73]:
# load data
data_dir = "data/"

college_players_stats = pd.read_csv(f'{data_dir}college_players_stats.csv', index_col=0)
drafted_players = pd.read_csv(f'{data_dir}drafted_players.csv', index_col=0)
not_drafted = pd.read_csv(f'{data_dir}not_drafted.csv', index_col=0)
players_stats = pd.read_csv(f'{data_dir}players_stats.csv', index_col=0)

# add label and normalize relevant values

players_stats['drafted'] = players_stats['name'].isin(drafted_players['name'])
players_stats['drafted'] = players_stats['drafted'].astype(int)

# TODO: add other metrics for training
df = players_stats[['field_pct', 'three_pct', 'free_pct', 'drafted']]
df.dropna(inplace=True)
df.head()

/var/folders/30/c46f3dw51w749hbqzgst91r40000gn/T/ipykernel_94256/3779834938.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,field_pct,three_pct,free_pct,drafted
0,0.414,0.414,0.529,0
1,0.619,0.619,0.788,0
2,0.346,0.346,0.611,0
3,0.473,0.473,0.641,0
4,0.544,0.544,0.482,0


In [49]:
np.unique(df['drafted'], return_counts=True)

(array([0, 1]), array([30843,  1017]))

# models

In [113]:
seed = 42
epochs = 250
lr = 0.01
batch_size = 32

# get data
x = df[['field_pct', 'three_pct', 'free_pct']].values
y = df['drafted'].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=seed)

np.unique(y_test, return_counts=True)

(array([0, 1]), array([5597,  201]))

In [94]:
# scikit learn logistic regression
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

# do logistic regression
logreg = LogisticRegression(
    random_state=seed,
).fit(x_train, y_train)
logreg.predict(x_test)
logreg.predict_proba(x_test)
logreg.score(x_test, y_test)

0.9644705070714039

In [ ]:
# gradient boost

In [114]:
# mlp
import tqdm

class MLP(torch.nn.Module):
    def __init__(self, hidden=16):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(3, hidden)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden, 1)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

dataset = torch.utils.data.TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1))

model = MLP()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for i in tqdm.trange(epochs):
    dl = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
    
    for x, y in dl:
        optimizer.zero_grad()
        output = model(x)

        loss = criterion(output, torch.tensor(y, dtype=torch.float32).reshape(-1, 1))
        loss.backward()
        optimizer.step()

  0%|          | 0/250 [00:00<?, ?it/s]/var/folders/30/c46f3dw51w749hbqzgst91r40000gn/T/ipykernel_94256/1148220239.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = criterion(output, torch.tensor(y, dtype=torch.float32).reshape(-1, 1))
100%|██████████| 250/250 [01:07<00:00,  3.70it/s]


In [117]:
res = model(torch.tensor(x_test, dtype=torch.float32))

# get loss
loss = criterion(res, torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1))
loss

tensor(0.1376, grad_fn=<BinaryCrossEntropyBackward0>)